<img src="../../shared/img/slides_banner.svg" width=2560></img>

# Over-Fitting and Cross-Validation

In [ ]:
import sys

sys.path.append("../../")

from shared.src import quiet
from shared.src import seed
from shared.src import style

In [ ]:
from pathlib import Path
import random

from IPython.display import HTML, Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import seaborn as sns
import scipy.stats

In [ ]:
sns.set_context("notebook", font_scale=1.7)

import shared.src.utils.util as shared_util

In [ ]:
def make_yerr(predictions, observations):
    errors = predictions - observations
    positive_errors = np.where(errors>0, errors, 0)
    negative_errors = np.where(errors<0, -errors, 0)
    return np.stack([positive_errors, negative_errors])


def make_error_plot(params, observation_df, y="std_mpg", x="std_weight", featurizers=None):
    f, ax = plt.subplots(figsize=(12, 12))
    ax.scatter(observation_df[x], observation_df[y], s=144, label="observations")
    xs = np.linspace(-3, 3, num=1000)
    ax.plot(xs, predict_linear_model(params, xs, featurizers),
            lw=4, color="C1", label="prediction function");

    predictions =  predict_linear_model(params, observation_df[x], featurizers)
    yerr = make_yerr(predictions, observation_df[y])
    ax.errorbar(observation_df[x], predictions,
                yerr=yerr, ecolor="r", elinewidth=4, ls="none", zorder=0, label="errors");
    ax.set_ylim(-3, 3); ax.legend();
    MSE = round(np.sum(np.square(yerr)) / len(observation_df), 1)
    ax.set_title(f"MSE = {MSE}")

    
def preds_plot(x, y, data, model_results, use_step=False, **plot_kwargs):
    
    if "color" not in plot_kwargs:
        plot_kwargs["color"] = "C1"
    if "lw" not in plot_kwargs:
        plot_kwargs["lw"] = 4
    
    xs = data[x]
    model_predictions = model_results.model.predict(model_results.params)

    sorted_xs = xs[np.argsort(xs)]
    sorted_predictions = model_predictions[np.argsort(xs)]

    sns.lmplot(x, y, data=data, lowess=True);
    f, ax = plt.gcf(), plt.gca(); f.set_size_inches((12, 12))
    if not use_step:
        ax.plot(sorted_xs, sorted_predictions, **plot_kwargs)
    else:
        ax.step(sorted_xs, sorted_predictions, **plot_kwargs)

        
def ic_compare(ic_tables, ic_name="LOO"):
    f, ax = plt.subplots(figsize=(12, 6))
    n = len(ic_tables)
    ax.errorbar(ic_tables[ic_name], range(n), xerr=1 * ic_tables["SE"], linestyle="none", lw=4,
                marker=".", markersize=24);
    ax.set_yticks(range(n)); ax.set_yticklabels(ic_tables.index.values);
    ax.set_xlabel(ic_name)

    
def plot_cross_validated_models(cv_results, MAP_result, data, featurizers=None,
                                x="std_mpg", y="std_weight",
                                x_range=(-2.5, 3), ylim=([-2, 2.25])):
    f, ax = plt.subplots(figsize=(12, 12))
    [ax.plot(np.linspace(*x_range),
        predict_linear_model(pd.Series(result.params), np.linspace(*x_range), featurizers),
        alpha=0.05, color="C1", lw=4) for result in cv_results];
    sns.regplot(x, y, data, lowess=True, ax=ax, scatter_kws={"zorder": 3})
    ax.plot(np.linspace(*x_range, num=1000),
        predict_linear_model(MAP_result.params, np.linspace(*x_range, num=1000), featurizers),
        alpha=1, color="k", lw=4, label="Prediction on Full Dataset");
    ax.set_ylim(ylim); ax.legend();
    return ax

In [ ]:
def smf_fit(formula, data, regularizer=None):
    model = smf.ols(formula, data=data)
    if regularizer is not None:
        if regularizer in ["l1", "lasso", "laplace"]:
            results = model.fit_regularized(L1_wt=1)
        elif regularizer == ["l2", "ridge", "normal"]:
            results = model.fit_regularized(L1_wt=0)
        elif 0 <= regularizer <= 1:
            results = model.fit_regularized(L1_wt=regularizer)
        else:
            raise ValueError(f"Unknown regularizer {regularizer}")
    else:
        results = model.fit()
    return results

In [ ]:
def predict_linear_model(params, x, featurizers=None):
    if featurizers is None:
        featurizers = [lambda x: 1, lambda x: x]  # intercept and linear term
    return sum(pd.Series(params.values) * pd.Series(f(x) for f in featurizers))

# Today, we will consider how to evaluate and compare models.

### $R^2$ and similar criteria derived from the likelihood on observed data are insufficient.

The $R^2$, or variance explained, was derived for models with a Normal likelihood term.
It corresponds to a normalized version of the log-likelihood of the data.

If we consider a different likelihood, e.g. Bernoulli,
we get a different quantity for the log-likelihood.

## For these metrics, the MLE is always the winner.

$$p(\text{params}\vert \text{data}) \propto p(\text{data}\vert \text{params})\cdot p(\text{params})$$

MAP methods maximize the posterior probability (on the left)
while MLE maximizes the likelihood (first term on the right side of the $\propto$).

So if we only care about log-likelihood,
the MLE will always look like the best model,
even though it doesn't incorporate the prior information.

## Fundamentally, the MLE parameters &#8220;explain&#8221; the data as best as possible.

They make the observed data seem _minimally random_ or _minimally suprising_.

Relative to all other choices of parameters,
they make the outcomes we observed _maximally likely_.

## But the goal of modeling isn't always just to explain data we've seen, it's to predict data we _haven't_.

Put another way,
you don't get any credit for coming up with some explanation after the fact
that makes sense of everything you've observed.

In fact,
if the explanation is too accurate,
it might be overly-complicated or fixated on irrelevant details.

Consider the following proverb, of
[immemorial origin](https://en.wikipedia.org/wiki/For_Want_of_a_Nail):

### For Want of a Nail
For want of a nail the shoe was lost.

For want of a shoe the horse was lost.

For want of a horse the rider was lost.

For want of a rider the message was lost.

For want of a message the battle was lost.

For want of a battle the kingdom was lost.

And all for the want of a horseshoe nail.

While it is certainly possible that a causal chain links the lost nail to the fall of the kingdom,
it seems more plausible that facts about the kingdoms participating in the war
are where the key causal factors lie --
the ones that can be used to determine which kingdoms will fall in the future, for example.

Or the below "mind-map"
which purports to "explain" and connect
everything from Hurricane Katrina (AD 2005) and the [First Council of Nicaea](https://en.wikipedia.org/wiki/First_Council_of_Nicaea) (AD 325)
to the HIV crisis and [the sinking of the _Lusitania_](https://en.wikipedia.org/wiki/Sinking_of_the_RMS_Lusitania).

In [ ]:
HTML("""<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Below is the QWorld map which took years in the making. The most important document you will ever come across. Please share &amp; spread awareness. It&#39;s time the people receive their deserved hidden truths. It&#39;s time to break free from tyranny &amp; enslavement. <a href="https://twitter.com/hashtag/Qanon?src=hash&amp;ref_src=twsrc%5Etfw">#Qanon</a> <a href="https://twitter.com/hashtag/TheGreatAwakening?src=hash&amp;ref_src=twsrc%5Etfw">#TheGreatAwakening</a> <a href="https://t.co/1KzyY9zwBk">pic.twitter.com/1KzyY9zwBk</a></p>&mdash; QNN: Qanon News Network (@realQNN) <a href="https://twitter.com/realQNN/status/1035943634540081155?ref_src=twsrc%5Etfw">September 1, 2018</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>""")

While this "model" does an admirable job of explaining many historical observations,
it is not very useful for predicting either additional, unobserved historical data or the future.

The map is based on the QAnon phenomenon,
which centered on "decoding" the bizarre and cryptic internet posts of an individual known as Q,
who claimed inside knowledge of a massive and bizarre conspiracy.

Inevitably, these posts would generate fevered speculation when they appeared,
then be used to "explain" major news events over the following days and weeks.
But they were effectively useless for predicting those events before they occurred.

The website
[spurious correlations](https://www.tylervigen.com/spurious-correlations)
collects examples of likelihood-maximizing linear models
that seem to clearly "miss the point" in these (and other) ways.

## The problem of explaining observed data &#8220;too well&#8221; is known as _over-fitting_.

We have fit, or explained, the data _so well_ that we've also explained
meaningless fluctuations we'd rather ignore.

## To adequately compare models, we must see how they perform on new data.

Obtaining new data to measure a model's performance is known as _validation_.

The technique commonly used to "fake" validation,
just as boot-strapping is used to "fake" resampling,
is known as _cross-validation_.

Over-fitting and cross-validation will be the topics of these slides.

# First, let's demonstrate over-fitting rigorously and quantitatively.

We'll use the `mpg` dataset,
which contains a bunch of measurements of cars produced in the 70s and 80s.

In [ ]:
mpg_df = sns.load_dataset("mpg", data_home=Path("..") / ".." / "shared" / "data")

In [ ]:
sns.pairplot(data=mpg_df);

## Not every relationship in this dataset can be modeled linearly.

For example, consider the relationship between weight and fuel efficiency (in _miles per gallon_).

In [ ]:
f, ax = plt.subplots(figsize=(12, 12))
sns.scatterplot(x="weight", y="mpg", data=mpg_df);

These two factors don't appear unrelated,
but they don't appear linearly related either.

First, we'll fit a linear model to their ($z$-scored) values to confirm this.

In [ ]:
def zscore(vals):
    return (vals - vals.mean()) / vals.std(ddof=1)

mpg_df["std_weight"] = zscore(mpg_df["weight"])
mpg_df["std_mpg"] = zscore(mpg_df["mpg"])

For now,
we'll just focus on maximum likelihood and maximum a posteriori methods,
and so we'll use `statsmodels`,
a Python library that allows us to write and "fit"
models using the formula interface.

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
weight_mpg_linear_ols_model = smf.ols(  # ordinary least squares, aka Normal likelhood, Flat priors
    "std_mpg ~ std_weight", data=mpg_df)

This formula says that the (standardized) fuel efficiency is Normally distributed
around a line computed from the (standardized) weight
with some slope and intercept.

By running the `.fit()` method,
we effectively execute `find_MAP`
in the equivalent pyMC model.

We also obtain some null hypothesis statistical tests
on that model,
checking whether the parameters are "significantly" different from 0.

In [ ]:
weight_mpg_linear_ols_results = weight_mpg_linear_ols_model.fit()

print(weight_mpg_linear_ols_results.rsquared)

weight_mpg_linear_ols_results.summary()

The output of this method is rich --
it's intended for professional statisticians.

We're focused only on the value of $R^2$,
the cell **R-squared** in the top-right.

The $R^2$ is high, relatively, but a high $R^2$ doesn't mean the job of prediction is done.

Just as we should always view our data before we start doing modeling and statistics,
we should always view the predictions of our model before we start drawing conclusions from it.

In [ ]:
params = weight_mpg_linear_ols_results.params
f, ax = plt.subplots(figsize=(12, 12))
sns.regplot("std_weight", "std_mpg", data=mpg_df, lowess=True, color="C0",
            scatter_kws={"alpha": 0.8}, line_kws={"lw": 4, "label": "Smooth Prediction", "color": "k"})
ax.plot([-1.5, 2.5], params["std_weight"] * pd.Series([-2.5, 2.5]) + params["Intercept"],
        color="C1", lw=4, label="Best Linear Prediction");
ax.legend();

The black line, labeled "Smooth Prediction", is produced by a method called LOWESS:
[LOcally-WEighted Scatterplot Smoothing](https://www.statisticshowto.datasciencecentral.com/lowess-smoothing/).

It can be used as a regression model,
but it is more commonly used as a visualization tool:
a way to visually estimate the (possibly non-linear) relationship between two variables,
as it is being used here.

It is closely related to the regression methods covered in data8.

Notice how the linear predictions seem to be under-estimating the mileage for low and high weights,
while over-estimating the mileage for intermediate weights?

This might be more visible in the plot below,
which visualizes the prediction errors as red bars
connecting the model prediction to the actual observations.

In [ ]:
make_error_plot(params, mpg_df.sample(frac=0.25))

Alternatively,
we might visualize our errors as a function of the independent variable,
`std_weight`,
to look for patterns.

In [ ]:
errors = (params["Intercept"] + params["std_weight"] * mpg_df["std_weight"]) - mpg_df["std_mpg"]

In [ ]:
f, ax = plt.subplots(figsize=(12, 12))
ax.scatter(mpg_df["std_weight"], errors); ax.set_ylim(-3, 3);
ax.hlines(0, -3, 3, lw=4); ax.set_xlim(-3, 3)
ax.set_xlabel("std_weight"); ax.set_ylabel("Error Size");

For the low and high weights,
the errors are primarily negative (under-estimation)
while for intermediate weight, the errors are primarily positive (over-estimation).

# One solution is to transform the data so that the relationship becomes linear.

This is known as _linearization_.

A linearized model with Normal likelihood using
functions `f` to transform the data looks like:

$$
y \sim \text{Normal}\left(\sum\text{slope}_i\cdot \texttt{f[i]}(x), \sigma\right)
$$

In [ ]:
def predict_linear_model(params, x, featurizers=None):
    if featurizers is None:
        featurizers = [lambda x: 1, lambda x: x]  # intercept and linear term
    return sum(pd.Series(params.values) * pd.Series(f(x) for f in featurizers))

While this freedom may seem strange,
note that there's not necessarily anything "natural" about the form our data comes to us:
we could've used "gallons per mile" instead of "miles per gallon",
which would correspond to using `1/x` as our featurizer.

# Linearizing with Categories

We've seen a version of data featurization with categorical linear models.

Often, a "category" or "group" is just a discretization of an underlying continuous value.

The function `binify` below adds an extra column to a dataframe
that has `nbins` different discrete category identities for the provided `column`.
It uses a pandas function, `pd.cut`, to define the bins.

In [ ]:
def binify(df, nbins, column="weight"):
    raw_bin_column = column + "_raw_bin"
    df[raw_bin_column] = pd.cut(df[column], bins=nbins)

    mapper = {k:v for k, v in zip(sorted(df[raw_bin_column].unique()), range(nbins))}

    df[column + "_bin"] = df[raw_bin_column].apply(lambda k: mapper[k])

If we apply it to the `std_weight` column,
we get a new categorical variable, `std_weight_bin`.

In [ ]:
binify(mpg_df, nbins=25, column="std_weight")

mpg_df[["std_weight", "std_weight_raw_bin", "std_weight_bin"]].sample(10)

We can also visualize the resulting data,
using `hue` to indicate the bins.

In [ ]:
f, ax = plt.subplots(figsize=(12, 12))
ax = sns.scatterplot(x="std_weight", y="std_mpg", hue="std_weight_bin",
               data=mpg_df, legend=False);
ax.set_xlim([-3, 3]); ax.set_ylim([-3, 3]);

We can then propose a model that applies a separate linear regression
to each bin.

In [ ]:
bin_model = smf.ols("std_mpg ~ std_weight * C(std_weight_bin)", data=mpg_df)

We might alternatively propose a strictly categorical model for each bin.

Uncomment and run the cell below to fit that model instead.

In [ ]:
# bin_model = smf.ols("std_mpg ~ (std_weight_bin)", data=mpg_df)

In [ ]:
bin_model_results = bin_model.fit()

print(bin_model_results.rsquared, weight_mpg_linear_ols_results.rsquared)

bin_model_results.summary()

As the number of bins, and so the number of parameters, increases,
the length of the output increases.
But we remain only interested in `rsquared`.
Increasing `nbins` increases `rsquared`.
With a sufficient number of bins,
we should always be able to outperform the baseline linear regression model,
even if we use the pure cetegorical model.

For technical reasons,
keep `nbins` below ~30.

For two or three bins,
the predictions of the model look reasonable:

In [ ]:
preds_plot("std_weight", "std_mpg", mpg_df, bin_model_results)

Go back and increase the value of `nbins` in `binify` to `25` (from the original `2`).

`rsquared` will increase,
but the predictions will become _less sensible_:
like a conspiracy theorist, the maximum likelihood model
sees complicated, intricate patterns that are probably not real.

This method is known as _piecewise linear regression_:
we fit a linear regression model to each "piece" of the input range.

You might think of it as the "regression version" of a histogram.

Where a histogram estimates a distribution by putting points into bins and then counting them up,
this model estimates the relationship between two values by putting the inputs into bins
and then fitting a regression model to the outputs inside each bin.

## In the extreme, we might consider making a new bin for every value we observe.

These kinds of models are known as _nearest neighbor_ models.

The code below implements nearest neighbor prediction
so that it can be used in concert with `statsmodels`
and so that the connection to linear models is clear.

Effectively,
we are picking a very complex "featurizer" `f`,
the function `nearest_neighbors`.
It is essentially a
[look-up table](https://en.wikipedia.org/wiki/Lookup_table)
based on our observed data.

In [ ]:
def make_nn_formula(exemplars):
    return "std_mpg ~ " + "-1 + nearest_neighbors(std_weight)"

nn_formula = make_nn_formula(mpg_df["std_weight"])

In [ ]:
# like a dictionary whose keys are weights and values are corresponding mpgs
mapper = pd.Series(index=mpg_df["std_weight"].values, data=mpg_df["std_mpg"].values)

neighbors = mpg_df["std_weight"].values

def nearest_neighbor_pred(x):
    # how far is x from each point we observed?
    squared_differences = np.square(neighbors - x)
    # what are the indices of the closest points? (who are my "nearest neighbors"?)
    nearest_value_indices = np.where(squared_differences == squared_differences.min())
    # what are the values at those indices?
    nearest_values = list(neighbors[nearest_value_indices])
    # what are the y values for my neighbors?
    neighbor_ys = mapper[nearest_values]
    return np.mean(neighbor_ys)

def nearest_neighbors(xs):
    return pd.Series(nearest_neighbor_pred(x) for x in xs)

The result is a model that can beat the pants off of any other in terms of `rsquared`:

In [ ]:
nn_model = smf.ols(nn_formula, data=mpg_df)

nn_model_results = nn_model.fit()

print(nn_model_results.rsquared, weight_mpg_linear_ols_results.rsquared)

nn_model_results.summary()

But if we look at the predictions, we can see an obvious problem:
the model is designed to pass exactly through each point in the dataset, if possible.

In [ ]:
preds_plot("std_weight", "std_mpg", mpg_df, nn_model_results, use_step=True,
           zorder=0, lw=2)

While this does a good job, by construction,
predicting the values we observed,
it does a terrible job of uncovering the true relationship between the variables.

The model is a slave of the dataset, forced to recreate every single variation,
even though we wouldn't expect to be able to perfectly predict the mileage of a car just from its weight.
There are too many other contributing factors that we are ignoring.

Pretending that those factors don't exist and that the input data
has all the information we need and trying to force a model
to achieve near-perfect performance on that data
is a major cause of over-fitting.

The categorization and nearest-neighbor models
demonstrate the principle of over-fitting.

Polynomial models are also commonly used to demonstrate over-fitting.

# Another Approach to Linearization: Polynomial Models

Any function can be approximated as a weighted sum of polynomial (power) functions:
$$
f(x) \approx a + b\cdot x + c\cdot x^2 + d\cdot x^3 \dots
$$

This is known as a
[Taylor Expansion](https://www.khanacademy.org/math/ap-calculus-bc/bc-series-new/bc-10-14/v/visualizing-taylor-series-approximations),
which you might have come across in a calculus class.

By incorporating polynomial functions into our formula,
we can find the values for $a$, $b$, $c$ etc. that minimize the prediction error.

In [ ]:
def make_poly_formula(y, x, max_degree):
    return y + " ~ " + " + ".join(
        ["-1"] + [f"np.power({x}, {k})" for k in range(max_degree + 1)])

max_degree = 15  # largest power of x to include in data features
polyformula = make_poly_formula("std_mpg", "std_weight", max_degree)
polyformula

In [ ]:
def make_polynomial_featurizers(max_degree=1):
    featurizers = []
    for k in range(max_degree + 1):
        def featurizer(x, k=k):
            return np.power(x, k)
        featurizers.append(featurizer)
    return featurizers

poly_featurizers = make_polynomial_featurizers(max_degree)

We just need to pass the formula to `smf.ols`.

In [ ]:
poly_model = smf.ols(polyformula, data=mpg_df)

poly_model_results = poly_model.fit()

print(poly_model_results.rsquared, weight_mpg_linear_ols_results.rsquared)

poly_model_results.summary()

Notice that `rsquared` has gone up, relative to the linear model.

But the model isn't doing a much better job.

In [ ]:
preds_plot("std_weight", "std_mpg", mpg_df, poly_model_results)

For `max_degree=5`,
notice the upwards curve all the way in the bottom right:
It's extremely unlikely that the true relationship between weight and mileage
includes that "kink",
but because it is in the dataset and our model was able to "predict" it,
it is included in our final prediction function.

Increase `max_degree` to `30`
(any higher and `smf.fit` starts to break down for numerical reasons).

`rsquared` will increase a bit,
but the prediction function will become even wonkier.

# $R^2$ is a flawed metric if measured on data the model got to observe before or during the fitting process.

It's too easy to "game" this metric,
unintentionally or intentionally.

We want to know how well a model performs for data it hasn't observed:
_out-of-sample_ data.

# Ideally, we'd go out and collect a bunch of new data once we finished fitting our model.

This is called _validating_ the model.

But this isn't always practical:
if we've run an experiment, we don't want to have to run it again.

# One practical solution is to change our fitting process so that the model doesn't get to see all of the data.

### And the most common version of this is called _cross-validation_.

In cross-validation, the data is split into two pieces:
one set for picking parameters with MLE/MAP or getting the posterior
and the other set for checking $R^2$ (or a similar metric).

These are called the _training_ and _testing_ sets:
one is used to "train" the model,
improving its performance, much like an athlete trains,
while the other is used to "test" its performance.

### While this isn't too hard to implement yourself, we'll use a pre-written version.

It comes from the [`sklearn` (scikit-learn) library](https://scikit-learn.org/stable/),
which is used to do maximum likelihood and maximum a posteriori fitting
using the terminology and idiom of machine learning,
just as `statsmodels` does the same
using the terminology and idiom of frequentist linear modeling.
Both of course do other things.

scikit-learn is more fully fleshed out, in certain ways,
than is `statsmodels`,
since the audience for traditional methods
is captured by the R statistical programming language,
while the audience for machine learning skews towards Python.

The [documentation](https://scikit-learn.org/stable/documentation.html)
and [examples](https://scikit-learn.org/stable/auto_examples/index.html)
for scikit-learn are a great resource for learning more about the machine learning side of data science.

In [ ]:
import sklearn.model_selection

The code cells below demonstrate the train/test split function on a small, simple dataframe.

In [ ]:
example_df = pd.DataFrame({"x":[1, 2, 3, 4, 5], "y":["A", "E", "I", "O", "U"]})
example_df

We can change the size of the `test` set by changing the `test_size` argument.

If `test_size` is an integer, it sets the total number of examples in the `test` set.
If it's a float between 0 and 1, it sets the relative size of the `test` set.

In [ ]:
train, test = sklearn.model_selection.train_test_split(example_df, test_size=2)
print("TRAIN:\n", train)
print("TEST:\n", test)

## Cross-validation involves splitting the data into train and test sets many times.

We want to see how well 

In [ ]:
num_splits = 1000

cross_validation_splits = [sklearn.model_selection.train_test_split(mpg_df, test_size=100)
                           for _ in range(num_splits)]
training_sets, test_sets = zip(*cross_validation_splits)  # "unzip"

Technical note: `cross_validation_splits` is a list of tuples,
where the first entry in the tuple is the training set,
while the second is the test set.

The last line above converts `cross_validation_splits` into a tuple of lists,
where the first entry is a list of training sets, while the second is a list of test sets,
and assigns them to appropriate names.

The cell below shows what a subset of the test sets (gold) look like
compared to their respective training sets (blue).

In [ ]:
n_examples = 4
f, axs = plt.subplots(figsize=(12, 6), nrows=2, ncols=n_examples, sharex=True, sharey=True)

axs[0, 0].set_ylabel("Training Sets"); axs[1, 0].set_ylabel("Test Sets")
for ax, training_set in zip(axs[0, :], training_sets[:n_examples]):
    ax.scatter("std_mpg", "std_weight", data=training_set, alpha=0.25)
for ax, test_set in zip(axs[1, :], test_sets[:n_examples]):
    ax.scatter("std_mpg", "std_weight", data=test_set, color="C1")

Instead of thinking of this as a method for obtaining "fake new data",
you might think of it as a way of disrupting certain patterns in the dataset.

If you focus on the top row, the training sets,
you'll see that for the intermediate values,
the relationship between the two variables looks about the same every time.
At the edges, where data is scarcer,
the location of the best fit line changes from split to split.

Similarly, if you focus on the bottom row, the test sets,
you'll see that the patterns at the edges of the dataset vary greatly.
A model that obtains high performance by over-fitting to a pattern present there in the training set
may fail miserably when that pattern fails to appear in the test set.

## We apply the same fitting procedure to each training set as though it were the original dataset we had observed.

In [ ]:
cv_results_poly = [smf_fit(polyformula, training_set) for training_set in training_sets]

We can then look at the predictions of each of the fitted models
and compare them to each other and to the original datasaet.

In [ ]:
plot_cross_validated_models(cv_results_poly, poly_model_results, data=mpg_df, featurizers=poly_featurizers);

While the predictions remain fairly stable in the intermediate region
across cross validation splits (gold, transparent),
the predictions at the edges vary wildly.

If you look closely, you'll see some predictions in that region that are wildly off:
they don't even come close to the observed data.
When the model is allowed to see the entire dataset,
this is prevented by the Normal likelihood,
which penalizes such large squared errors.

But when the model is forced to work off of only what it observes in the training set,
there is nothing to prevent these fluctuations.

Put another way,
one of the consequences of a model developing "conspiracy theories"
about the data in order to fit it better
is instability and poor performance on data it did not observe.
The more complex patterns, though they match the observed data perfectly,
make very poor predictions on unobserved values.

## Then, we take the fitted model parameters and evaluate $R^2$ on the test data.

Subsets of data withheld from the model, like the test sets above,
are also known as _held-out_ datasets.

This "cross-validated $R^2$" function is not any different from
any other function for calculating $R^2$ --
the only thing that is changed is the names of the variables.

In [ ]:
def cross_validated_r_squared(heldout_data, cv_result, featurizers=None):
    heldout_x = heldout_data["std_weight"]
    heldout_y = heldout_data["std_mpg"]
    predictions_on_heldout = predict_linear_model(cv_result.params, heldout_x, featurizers)
    
    return 1 - ((predictions_on_heldout - heldout_y) ** 2).mean() / heldout_y.var(ddof=0)

In [ ]:
cv_r_squareds_poly = pd.Series([
    cross_validated_r_squared(test_set, cv_result, featurizers=poly_featurizers)
     for test_set, cv_result in zip(test_sets, cv_results_poly)])

The $R^2$ values on the test sets confirm our concerns:
the $R^2$ on unseen data may be negative!

In [ ]:
f, ax = plt.subplots(figsize=(12, 6))
ax.hist(cv_r_squareds_poly, range=(-10, 1), # this distribution has a long negative tail
        bins=100, label="High Degree Polynomial", density=True);
ax.set_xlabel("$R^2$ on Test"); ax.set_xlim([-2, 1]); ax.legend();
cv_r_squareds_poly.mean()

The mean is printed to indicate the existence of the long negative tail:
its value is typically to the left-most edge of the plotted region,
even though the bulk of the values are close to the right-most edge.

This means that there is a small number of very large negative values,
pulling the average down.

## If we examine the worst-performing models, we can see where the long tail comes from.

In [ ]:
worst_performers = cv_r_squareds_poly.sort_values().head(10)  # which models had lowest R^2

In [ ]:
idx = worst_performers.index[-1]
make_error_plot(cv_results_poly[idx].params, test_sets[idx], featurizers=poly_featurizers);

`run_cross_validation` below
wraps the cross-validation process up into a single function.

In [ ]:
def run_cross_validation(data, formula, num_folds=1000, test_size=100, featurizers=None):
    # split the data into training and testing sets num_folds times
    cross_validation_folds = [sklearn.model_selection.train_test_split(
        data, test_size=test_size) for _ in range(num_folds)]
    training_sets, test_sets = zip(*cross_validation_folds)
    
    # fit same model to each training set
    results = [smf_fit(formula, training_set) for training_set in training_sets]
    
    # use corresponding test set to get r_squared for each model
    r_squareds = pd.Series([
        cross_validated_r_squared(test_set, result, featurizers=featurizers)
        for test_set, result in zip(test_sets, results)
    ])
    return results, r_squareds

# Models without the flexibility to over-fit will generally have less variability in their $R^2$ values on unobserved data.

In [ ]:
cv_results_linear, cv_r_squareds_linear = run_cross_validation(mpg_df, "std_mpg ~ std_weight")

In [ ]:
plot_cross_validated_models(cv_results_linear, weight_mpg_linear_ols_results, data=mpg_df);

In [ ]:
f, ax = plt.subplots(figsize=(12, 6))
ax.hist(cv_r_squareds_poly, range=(-10, 1), bins=100, label="High Degree Polynomial",
        density=True, histtype="step", lw=4);
ax.hist(cv_r_squareds_linear, bins=10, label="Linear",
        density=True, histtype="step", lw=4);
ax.set_xlabel("$R^2$ on Test"); ax.set_xlim([-2, 1]); ax.legend(); cv_r_squareds_linear.mean()

# We can easily perform a version of cross-validation with pyMC.

But note that Bayesian approaches to model comparison and cross-validation are still in their relative infancy.
As with many things Bayesian,
there is a rigorous answer on how to perform model comparsion,
given by Bayes' rule,
but it is onerous, even impractical, to compute,
necessitating clever approximations.

The approximate method in these slides was published in 2015.

But first,
let's produce some models and posteriors.

As a baseline, just like in $R^2$, we choose the constant model.

In [ ]:
with pm.Model() as constant_model_pymc:
    pm.GLM.from_formula("std_mpg ~ 1", mpg_df)

In [ ]:
with constant_model_pymc:
    constant_model_trace = pm.sample()

constant_model_posterior_df = shared_util.samples_to_dataframe(constant_model_trace)

constant_model_MAP = pm.find_MAP(model=constant_model_pymc)

constant_model_MAP

We also do a plain linear model.

In [ ]:
with pm.Model() as linear_model_pymc:
    pm.GLM.from_formula("std_mpg ~ std_weight", mpg_df)

In [ ]:
with linear_model_pymc:
    linear_model_trace = pm.sample()

linear_model_posterior_df = shared_util.samples_to_dataframe(linear_model_trace)

linear_model_MAP = pm.find_MAP(model=linear_model_pymc)

linear_model_MAP

In [ ]:
pm.plot_posterior(linear_model_trace, figsize=(12, 12), text_size=16, ref_val=[0, 0, 1]);

A polynomial model.

If you try to increase the `max_degree` too much,
e.g. to 10 or more,
you'll find that sampling slows to a crawl and you start to see warnings from pyMC.

This is an indication that the model is very bad:
for the very high degree polynomial models,
the posterior is a gnarly function.

In some ways,
this is a feature,
rather than a bug,
because it means that
we are prevented from making a silly model
like the super-high-degree polynomial.

But not that this means that below,
we won't be working with the same
polynomial model to which we applied the over-fitting analysis.

In [ ]:
polyformula = make_poly_formula("std_mpg", "std_weight", max_degree=5)

with pm.Model() as poly_model_pymc:
    pm.GLM.from_formula(polyformula, mpg_df)

In [ ]:
with poly_model_pymc:
    poly_model_trace = pm.sample()

poly_model_posterior_df = shared_util.samples_to_dataframe(poly_model_trace)

poly_model_MAP = pm.find_MAP(model=poly_model_pymc)

poly_model_MAP

In [ ]:
pm.plot_posterior(poly_model_trace, figsize=(8, 16), text_size=16, ref_val=[0] * 6 + [1]);

A piecewise linear-regression model.

Again, we are held back from using too many bins
by the fact that sampling will begin to fail.
Again, this is somewhere in between a bug and a feature
of Bayesian Monte Carlo methods.

In [ ]:
binify(mpg_df, nbins=5, column="std_weight")

with pm.Model() as bin_model_pymc:
    pm.GLM.from_formula("std_mpg ~ std_weight*C(std_weight_bin)", mpg_df)

In [ ]:
with bin_model_pymc:
    bin_model_trace = pm.sample()

bin_model_posterior_df = shared_util.samples_to_dataframe(bin_model_trace)

bin_model_MAP = pm.find_MAP(model=bin_model_pymc)

bin_model_MAP

## Once we've obtained a posterior, we can get an estimate of cross-validation performance effectively for free.

What we'd naïvely need to do is perform a version of cross-validation
where we'd apply `pm.sample` to the training set
and then compute the average negative log-likelihood
on the test set.

We'd need to then _also_ average this value over the posterior for the parameters,
given the observations in the training set.

This would take a very, very long time.

However, the results can be easily estimated from the trace output by `pm.sample`,
at least for the case where the test sets have size `1` (only a single datapoint is left out).

The abbreviation for this type of cross-validation is `LOO`, or `L`eave `O`ne `O`ut.

The details of how are unimportant and technical.

First, we need to organize our traces.

The cell below gives each model a name and puts the models and traces into the format
expected by pyMC's model comparison function, `compare`.

It is a dictionary whose keys are the models and whose values are the traces.

In [ ]:
constant_model_pymc.name = "constant: y ~ 1"
linear_model_pymc.name = "linear: y ~ 1 + x"
poly_model_pymc.name = "poly: y ~ 1 + x + x^2 ..."
bin_model_pymc.name = "bin: y ~ 1 + in_bin_0(x)*x + ..."

model_traces = {constant_model_pymc: constant_model_trace,
               linear_model_pymc: linear_model_trace,
               poly_model_pymc: poly_model_trace,
               bin_model_pymc: bin_model_trace}

Then we pass that dictionary to the `pm.compare` function.

The `ic` keyword tells the function which criterion to use.
Here, we choose the leave-one-out cross-validation criterion.

Due to connections to information theory,
these model comparison criteria are known as `i`nformation `c`riteria
in the world of Bayesian inference.

In [ ]:
ics = pm.compare(model_traces, ic="LOO")  # L(eave) O(ne) O(ut) Cross-Validation

ics

Again, the output is quite rich,
but we only care about a small subset of it.

The `LOO` column is the estimate of the performance on unseen data.
A lower value is better.

The `SE` column indicates the uncertainty in that estimate,
in terms of standard error.

The `shape_warn` column will include a `1` if the approximation step
failed.

The values are compared the same way
we compare any other univariate value with an estimate and a standard error.

In [ ]:
ic_compare(ics, ic_name="LOO")

Lower is better,
and so we can conclude that the `poly`, `bin`, and `linear` models
are better than the `constant` model.

The high overlap of the standard errors indicates that there's not much evidence
to favor any model among those three.

In theory, these criteria take complexity into account,
but they tend to be insufficiently conservative.
Ockham's razor, then, would suggest using the linear model.

Note: the older alternative to LOO is called the
[Weighted Akaike Information Criterion](https://en.wikipedia.org/wiki/Akaike_information_criterion),
or `WAIC`.
If you look into Bayesian model comparison,
you'll see more on the WAIC and its variants than on LOO,
due to its relative novelty.

If you change the `"LOO"` arguments above to `"WAIC"`,
you'll see the results of comparing the models with `"WAIC"`.
They are substantially similar,
since the two are closely related.

# But we can do better.

Let's return to the original problem.

We wanted to see how fuel efficiency and weight related to one another.

We measured fuel efficiency in miles per gallon.

But weight should be _directly_ related to the gallons required to drive a mile,
meaning it's _inversely_ related to the miles driven on a single gallon.

In [ ]:
mpg_df["gpm"] = 1 / mpg_df["mpg"]  # gallons per mile is 1 / miles per gallon
mpg_df["std_gpm"] = zscore(mpg_df["gpm"])  # standardize after transforming

After we apply that transformation to our output,
the relationship between the variables appears much more linear:

In [ ]:
sns.lmplot(y="std_gpm", x="std_weight", data=mpg_df, height=12);

If we apply our Bayesian modeling and model comparison tools,
we see that this model wins out.

In [ ]:
with pm.Model() as inv_model_pymc:
    pm.GLM.from_formula("std_gpm ~ std_weight", mpg_df)

with inv_model_pymc:
    inv_model_trace = pm.sample()

inv_model_posterior_df = shared_util.samples_to_dataframe(inv_model_trace)

inv_model_MAP = pm.find_MAP(model=inv_model_pymc)

inv_model_MAP

In [ ]:
inv_model_pymc.name = "inverse: 1/y ~ x"
model_traces = {constant_model_pymc: constant_model_trace,
               linear_model_pymc: linear_model_trace,
               poly_model_pymc: poly_model_trace,
               bin_model_pymc: bin_model_trace,
               inv_model_pymc: inv_model_trace}

In [ ]:
ics = pm.compare(model_traces, ic="LOO")  # L(eave) O(ne) O(ut) Cross-Validation

ics

In [ ]:
ic_compare(ics)

There is still some overlap between the standard errors of the polynomial/bin models and the inverse model,
indicating there's a small chance that those models have better out-of-sample performance.

If we include our strong preference for simpler models, though,
we can come down in favor of the inverse model.